In [213]:
import numpy as np

## Input

In [214]:
ymax = 3.5
t = 0.5
delta = 0.5
S = np.array([1, 2, 3, 4])
R = np.array([[0, 1, 0, 2],
              [1, 0, 4, 5],
              [1, 2, 0, 1],
              [9, 2, 3, 0]])
pi = np.array([0.5, 0,4, 0.1, 0])
r = np.array([2,2,4,5])
epsilon = 10e-4

In [215]:
epsilon

0.001

In [216]:
def create_Q_matrix(R):
    length = len(R)
    Q = np.zeros((length, length))
    
    for i in range(0, length):
        R[i][i] = -1 * np.sum(R[i][:])
        
    return R

In [217]:
Q = create_Q_matrix(R)

In [218]:
Q

array([[ -3,   1,   0,   2],
       [  1, -10,   4,   5],
       [  1,   2,  -4,   1],
       [  9,   2,   3, -14]])

In [219]:
def create_D_matrix(r):
    D = np.diag(r)
    return D

In [220]:
D = create_D_matrix(r)

In [221]:
D

array([[2, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 4, 0],
       [0, 0, 0, 5]])

In [222]:
def create_Q_transition_matrix(reward, dy, num_state, Q, D):
    new_transition = int(reward / dy)
    size_c = new_transition*num_state
    C = np.zeros((size_c, size_c))
    C = np.asmatrix(C)
    size_q = len(Q)
    size_d= len(D)
    
    for i in range(0, new_transition):
        idx = i * num_state
        if i == 0:
            C[idx:size_q, idx:size_q] = Q
            C[idx:size_q, idx + size_q:idx + size_d +size_d] = D/dy
        else:
            C[idx:size_q + idx, idx:size_q + idx] = Q
            
            if idx != (C.shape[0] - size_q):
                C[idx:size_q + idx, idx+size_q:idx+size_q+size_q] = D/dy
            
    sum_C_diagonal = -C.sum(axis=1)

    main_diagonal_values = []
    for i in range(0, num_state):
        tmp = 0
        if C[i,i] < 0:
            tmp =  sum_C_diagonal[i] + C[i,i]
        else:
            tmp = sum_C_diagonal[i] - C[i,i]
        C[i,i] = tmp
        main_diagonal_values.append(tmp)
    
    for i in range(num_state, size_c):
        C[i,i] = main_diagonal_values[i % num_state]
    
    return C

In [223]:
dy = 0.5
reward = 3.5
num_state = 4
C = create_Q_transition_matrix(reward=reward, dy=dy, num_state=num_state, Q=Q, D=D)

In [224]:
C.shape[0]

28

In [225]:
with fullprint():
    print(C)

[[ -7.   1.   0.   2.   4.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1. -14.   4.   5.   0.   4.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   2. -12.   1.   0.   0.   8.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  9.   2.   3. -24.   0.   0.   0.  10.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  -7.   1.   0.   2.   4.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1. -14.   4.   5.   0.   4.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   2. -12.   1.   0.   0.   8.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  

In [226]:
from tempfile import TemporaryFile
outfile = TemporaryFile()

In [227]:
s = np.savetxt('test.out', C, delimiter=',')

In [228]:
D = np.asmatrix(D)

In [229]:
s

In [231]:
D

matrix([[2, 0, 0, 0],
        [0, 2, 0, 0],
        [0, 0, 4, 0],
        [0, 0, 0, 5]])

In [232]:
D[0:4,0:4] = Q

In [233]:
Q

array([[ -3,   1,   0,   2],
       [  1, -10,   4,   5],
       [  1,   2,  -4,   1],
       [  9,   2,   3, -14]])

In [234]:
D

matrix([[ -3,   1,   0,   2],
        [  1, -10,   4,   5],
        [  1,   2,  -4,   1],
        [  9,   2,   3, -14]])

In [235]:
class fullprint:
    'context manager for printing full numpy arrays'

    def __init__(self, **kwargs):
        kwargs.setdefault('threshold', np.inf)
        self.opt = kwargs

    def __enter__(self):
        self._opt = np.get_printoptions()
        np.set_printoptions(**self.opt)

    def __exit__(self, type, value, traceback):
        np.set_printoptions(**self._opt)

In [236]:
from scipy.sparse import csr_matrix
def sparse(matrix):
    return csr_matrix(matrix)

In [237]:
C_sp = sparse(C)

In [238]:
C

matrix([[ -7.,   1.,   0.,   2.,   4.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  1., -14.,   4.,   5.,   0.,   4.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  1.,   2., -12.,   1.,   0.,   0.,   8.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  9.,   2.,   3., -24.,   0.,   0.,   0.,  10.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  -7.,   1.,   0.,   2.,   4.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,

In [239]:
CC=C_sp*0.2

In [240]:
print(CC)

  (0, 0)	-1.4000000000000001
  (0, 1)	0.2
  (0, 3)	0.4
  (0, 4)	0.8
  (1, 0)	0.2
  (1, 1)	-2.8000000000000003
  (1, 2)	0.8
  (1, 3)	1.0
  (1, 5)	0.8
  (2, 0)	0.2
  (2, 1)	0.4
  (2, 2)	-2.4000000000000004
  (2, 3)	0.2
  (2, 6)	1.6
  (3, 0)	1.8
  (3, 1)	0.4
  (3, 2)	0.6000000000000001
  (3, 3)	-4.800000000000001
  (3, 7)	2.0
  (4, 4)	-1.4000000000000001
  (4, 5)	0.2
  (4, 7)	0.4
  (4, 8)	0.8
  (5, 4)	0.2
  (5, 5)	-2.8000000000000003
  :	:
  (22, 20)	0.2
  (22, 21)	0.4
  (22, 22)	-2.4000000000000004
  (22, 23)	0.2
  (22, 26)	1.6
  (23, 20)	1.8
  (23, 21)	0.4
  (23, 22)	0.6000000000000001
  (23, 23)	-4.800000000000001
  (23, 27)	2.0
  (24, 24)	-1.4000000000000001
  (24, 25)	0.2
  (24, 27)	0.4
  (25, 24)	0.2
  (25, 25)	-2.8000000000000003
  (25, 26)	0.8
  (25, 27)	1.0
  (26, 24)	0.2
  (26, 25)	0.4
  (26, 26)	-2.4000000000000004
  (26, 27)	0.2
  (27, 24)	1.8
  (27, 25)	0.4
  (27, 26)	0.6000000000000001
  (27, 27)	-4.800000000000001
